In [24]:
import tensorflow as tf

# Load MNIST dataset from TensorFlow/Keras
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()

# Normalize the input images to the range [0, 1]
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

# Reshape the data to include the channel dimension (required for CNNs)
X_train = X_train.reshape((X_train.shape[0], 28, 28, 1))
X_test = X_test.reshape((X_test.shape[0], 28, 28, 1))

# Convert labels to categorical one-hot encoded vectors
y_train = tf.keras.utils.to_categorical(y_train, 10)  # 10 classes for digits 0-9
y_test = tf.keras.utils.to_categorical(y_test, 10)


11490434/11490434 [==============================] - 4s 0us/step


In [26]:
print(f'Train images shape: {X_train.shape}')
print(f'Train labels shape: {X_test.shape}')
print(f'Test images shape: {y_train.shape}')
print(f'Test labels shape: {y_test.shape}')

Train images shape: (60000, 28, 28, 1)
Train labels shape: (10000, 28, 28, 1)
Test images shape: (60000, 10)
Test labels shape: (10000, 10)


In [27]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

# Load and preprocess the MNIST dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# Normalize the input images to the range [0, 1]
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

# Reshape the data to include the channel dimension (required for CNNs)
# Assuming images are 28x28 pixels
X_train = X_train.reshape((X_train.shape[0], 28, 28, 1))
X_test = X_test.reshape((X_test.shape[0], 28, 28, 1))
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# Verify the shape of the data
print(f'Shape of X_train: {X_train.shape}')
print(f'Shape of y_train: {y_train.shape}')
print(f'Shape of X_test: {X_test.shape}')
print(f'Shape of y_test: {y_test.shape}')

# Define the CNN model
model = models.Sequential()

# First convolutional layer
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))

# Second convolutional layer
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Third convolutional layer
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

# Flatten the output from the convolutional layers
model.add(layers.Flatten())

# Fully connected layer
model.add(layers.Dense(64, activation='relu'))

# Output layer with softmax activation for classification
model.add(layers.Dense(10, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print the model summary
model.summary()

# Train the model
history = model.fit(X_train, y_train, epochs=20, batch_size=64, validation_split=0.2)

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Test accuracy: {test_acc:.4f}')


Shape of X_train: (60000, 28, 28, 1)
Shape of y_train: (60000, 10)
Shape of X_test: (10000, 28, 28, 1)
Shape of y_test: (10000, 10)
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 3, 3, 64)          3

In [28]:
model.save('mnist.h5')

In [2]:
from keras.models import load_model
import tkinter as tk
from PIL import ImageGrab, Image
import numpy as np
import matplotlib.pyplot as plt

# Load the trained model
model = load_model('mnist.h5')  # Ensure this matches the saved model

def predict_digit(img, save_path='processed_image.png'):
    # Resize image to 28x28 pixels
    img = img.resize((28, 28))
    # Convert RGB to grayscale
    img = img.convert('L')
    img = np.array(img)
    # Normalize the image
    img = img / 255.0
    # Reshape for model input
    img = img.reshape(1, 28, 28, 1)

    # Save the processed image for inspection
    plt.imsave(save_path, img.reshape(28, 28), cmap='gray')

    # Predict the class
    res = model.predict([img])[0]
    return np.argmax(res), max(res)

class App(tk.Tk):
    def __init__(self):
        tk.Tk.__init__(self)
        self.x = self.y = 0
        # Creating elements
        self.canvas = tk.Canvas(self, width=200, height=200, bg="black", cursor="cross")
        self.label = tk.Label(self, text="Analyzing..", font=("Helvetica", 48))
        self.classify_btn = tk.Button(self, text="Search", command=self.classify_handwriting)
        self.button_clear = tk.Button(self, text="Clear", command=self.clear_all)
        # Grid structure
        self.canvas.grid(row=0, column=0, pady=2, sticky=tk.W)
        self.label.grid(row=0, column=1, pady=2, padx=2)
        self.classify_btn.grid(row=1, column=1, pady=2, padx=2)
        self.button_clear.grid(row=1, column=0, pady=2)
        # Bindings
        self.canvas.bind("<B1-Motion>", self.draw_lines)

    def clear_all(self):
        self.canvas.delete("all")

    def classify_handwriting(self):
        self.update()
        # Correctly capture the canvas area
        x = self.winfo_rootx() + self.canvas.winfo_x()
        y = self.winfo_rooty() + self.canvas.winfo_y()
        x1 = x + self.canvas.winfo_width()
        y1 = y + self.canvas.winfo_height()
        print(f"Capture coordinates: ({x}, {y}, {x1}, {y1})")  # Debug print
        im = ImageGrab.grab(bbox=(x, y, x1, y1))
        im.save('captured_image.png')  # Save the captured image for debugging
        digit, acc = predict_digit(im)
        self.label.configure(text=str(digit) + ', ' + str(int(acc * 100)) + '%')

    def draw_lines(self, event):
        self.x = event.x
        self.y = event.y
        r = 8  # Brush size
        self.canvas.create_oval(self.x - r, self.y - r, self.x + r, self.y + r, fill='white')

app = App()
app.mainloop()


2024-06-25 20:57:44.332529: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-25 20:57:47.652417: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Capture coordinates: (5, 35, 211, 241)
1/1 [==============================] - 0s 180ms/step
Capture coordinates: (287, 65, 493, 271)
1/1 [==============================] - 0s 17ms/step
Capture coordinates: (287, 65, 493, 271)
1/1 [==============================] - 0s 15ms/step
Capture coordinates: (287, 65, 493, 271)
1/1 [==============================] - 0s 15ms/step
Capture coordinates: (287, 65, 493, 271)
1/1 [==============================] - 0s 15ms/step
Capture coordinates: (287, 65, 493, 271)
1/1 [==============================] - 0s 15ms/step
Capture coordinates: (287, 65, 493, 271)
1/1 [==============================] - 0s 15ms/step
Capture coordinates: (287, 65, 493, 271)
1/1 [==============================] - 0s 15ms/step
Capture coordinates: (287, 65, 493, 271)
1/1 [==============================] - 0s 15ms/step
Capture coordinates: (287, 65, 493, 271)
1/1 [==============================] - 0s 15ms/step
Capture coordinates: (287, 65, 493, 271)
1/1 [=========================